In [10]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width' , 1000)

In [11]:
# Variables
season = "2020-21"

In [12]:
all_players = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv' , parse_dates = ['kickoff_time'])
player_future_fixture = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])




In [13]:
all_teams.head()

,team_code,draw,form,team_id,loss,team_name,played,points,position,team_short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,NaN,1,0,Arsenal,0,0,0,ARS,4,NaN,False,0,1190,1270,1190,1200,1160,1270,1
1,7,0,NaN,2,0,Aston Villa,0,0,0,AVL,2,NaN,False,0,1050,1080,1010,1080,1000,1110,2
2,36,0,NaN,3,0,Brighton,0,0,0,BHA,2,NaN,False,0,1070,1070,1100,1130,1060,1060,131
3,90,0,NaN,4,0,Burnley,0,0,0,BUR,3,NaN,False,0,1140,1150,1130,1240,1050,1170,43
4,8,0,NaN,5,0,Chelsea,0,0,0,CHE,4,NaN,False,0,1200,1260,1140,1270,1240,1250,4


In [14]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,ep_this,event_points,form,player_id,in_dreamteam,news,news_added,now_cost,photo,PPG,selected_by_percent,status,team_id,team_code,total_points,...,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,PP90,PPMM,VAPM
0,NaN,NaN,37605,0,0,-1,1,0,3,-0.5,0.5,0,0.0,1,False,NaN,NaT,69,37605.jpg,0.0,0.9,a,1,3,0,...,0,0,0,0,0.0,0.0,0.0,0.0,536,217,537,219,534,215,537,218,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.289855
1,75.0,0.0,39476,-1,1,-1,1,0,2,-0.4,0.0,0,0.0,2,False,Calf injury - 75% chance of playing,2020-09-11 11:00:08.600094,49,39476.jpg,0.0,0.1,d,1,3,0,...,0,0,0,0,0.0,0.0,0.0,0.0,419,165,411,165,375,154,426,165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.408163
2,100.0,100.0,41270,0,0,0,0,0,2,0.0,1.0,1,0.5,3,False,NaN,2020-09-10 14:30:11.632386,55,41270.jpg,1.0,0.9,a,1,3,1,...,0,0,0,3,0.0,0.0,0.0,0.0,487,187,490,187,475,184,494,187,NaN,NaN,4.0,NaN,NaN,NaN,90.0,0.181818,-0.181818
3,NaN,NaN,54694,0,0,0,0,0,3,5.5,6.5,5,6.0,4,False,NaN,NaT,120,54694.jpg,6.0,36.6,a,1,3,12,...,0,0,0,39,64.6,51.7,69.0,18.5,31,15,19,15,16,6,13,9,NaN,NaN,6.0,NaN,1.0,NaN,6.0,0.500000,0.333333
4,75.0,NaN,58822,0,0,-1,1,0,2,-0.4,0.5,0,0.0,5,False,Calf injury - 75% chance of playing,2020-09-23 09:00:14.881983,49,58822.jpg,0.0,0.4,d,1,3,0,...,0,0,0,0,0.0,0.0,0.0,0.0,423,166,415,166,380,155,430,166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.408163


In [15]:
df_merge = all_teams[["team_code" , "strength" , "team_short_name" , "strength_overall_home" , "strength_overall_away"]]
all_players = all_players.merge(df_merge, on = 'team_code')

df_merge = player_types[['player_type_id' , 'position_name_short']]
all_players = all_players.merge(df_merge , on = "player_type_id")


#merged_data.drop(["element"], axis = 1, inplace = True)
#merged_data.rename(columns = {"fixture" : "fixture_id" , "opponent_team" : "opponent_team_id" , "round" : "gameweek_id" , "value" : "player_price", "short_name" : "opponent_team_name" , "strength" : "opponent_strength"}, inplace = True)

#merged_data.sort_values("gameweek_id" , ascending = True , inplace = True)



In [16]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'player_code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'player_type_id', 'ep_next', 'ep_this', 'event_points', 'form', 'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'PPG', 'selected_by_percent', 'status', 'team_id', 'team_code', 'total_points', 'transfers_in_event', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'threat_rank', 'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type', 'corners_and_indirect_freekicks_order', 'corners_and_indirect_freekicks_text', 'direct_freekicks_order',
     

In [17]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,ep_this,event_points,form,player_id,in_dreamteam,news,news_added,now_cost,photo,PPG,selected_by_percent,status,team_id,team_code,total_points,...,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,PP90,PPMM,VAPM,strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,NaN,NaN,37605,0,0,-1,1,0,3,-0.5,0.5,0,0.0,1,False,NaN,NaT,69,37605.jpg,0.0,0.9,a,1,3,0,...,0.0,0.0,0.0,536,217,537,219,534,215,537,218,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.289855,4,ARS,1190,1270,MID
1,NaN,NaN,54694,0,0,0,0,0,3,5.5,6.5,5,6.0,4,False,NaN,NaT,120,54694.jpg,6.0,36.6,a,1,3,12,...,51.7,69.0,18.5,31,15,19,15,16,6,13,9,NaN,NaN,6.0,NaN,1.0,NaN,6.000000,0.500000,0.333333,4,ARS,1190,1270,MID
2,NaN,NaN,84450,0,0,0,0,0,3,2.0,3.0,2,2.5,9,False,NaN,NaT,55,84450.jpg,2.5,0.8,a,1,3,5,...,23.9,21.0,6.8,133,47,74,49,92,49,88,44,NaN,NaN,2.0,NaN,NaN,NaN,2.694611,0.454545,0.090909,4,ARS,1190,1270,MID
3,NaN,NaN,195735,-1,1,-2,2,0,3,0.5,1.5,1,1.0,18,False,NaN,NaT,78,195735.jpg,1.0,1.2,a,1,3,2,...,17.6,14.0,3.4,257,115,97,64,123,69,184,86,1.0,NaN,3.0,NaN,2.0,NaN,4.390244,0.128205,-0.128205,4,ARS,1190,1270,MID
4,NaN,NaN,198849,0,0,-1,1,0,3,-0.5,0.5,0,0.0,19,False,NaN,NaT,49,198849.jpg,0.0,0.2,a,1,3,0,...,0.0,0.0,0.0,437,171,429,175,398,151,444,174,5.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.408163,4,ARS,1190,1270,MID


## Save to file

In [18]:
all_players.to_csv (r'./data/' + season + '/Merged_data/all_players_merged.csv', index = False, header=True)
